In [3]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'

# construct_training_set = True
construct_training_set = False

# use_cloudaugment = True
use_cloudaugment = False

# create U-Net folder
unet_model_dir = Path("../cloud_seg/models/unet/")
unet_model_dir.mkdir(parents=True, exist_ok=True)


In [4]:
# Things to add
# https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_cli.html

In [21]:
%%file ../cloud_seg/utils/augmentations.py
import albumentations as A
import numpy as np

class CloudAugmentations:
    def __init__(self, params):

        self.params = params

        # Dictionary to convert between abbreviation and full augmentation string                              
        self.aug_to_name = {
            'vf': 'VerticalFlip',
            'hf': 'HorizontalFlip',
            'rr': 'RandomRotate90',
            'nr': 'Normalize',
            # 'ss': 'SizeScale',
            # 'gb': 'GaussianBlur',
            # 'gn': 'GaussianNoise',
        }

        self.aug_to_func = {
            'vf': self.add_VerticalFlip,
            'hf': self.add_HorizontalFlip,
            'rr': self.add_RandomRotate90,
            'nr': self.add_Normalize,
            # 'ss': self.add_SizeScale,
            # 'gb': self.add_GaussianBlur,
            # 'gn': self.add_GaussianNoise,
        }
        
    # Augmentation functions are listed in the order that they (mostly) should be called   
    def add_VerticalFlip(self):
        self.augmentations_names.append(self.aug_to_name['vf'])
        self.augmentations.append(A.VerticalFlip(p=0.5))

    def add_HorizontalFlip(self):
        self.augmentations_names.append(self.aug_to_name['hf'])
        self.augmentations.append(A.HorizontalFlip(p=0.5))

    def add_RandomRotate90(self):
        self.augmentations_names.append(self.aug_to_name['rr'])
        self.augmentations.append(A.RandomRotate90(p=0.5))
        
    def add_Normalize(self):
        self.augmentations_names.append(self.aug_to_name['nr'])
        self.augmentations.append(
            A.Normalize(
                mean=self.params['band_means'],
                std=self.params['band_stds'],
                max_pixel_value=1.0,
                p=1.0,
            )
        )        
    def add_augmentations(self, augs_manual: str=None):
        self.augmentations = []
        self.augmentations_names = []

        # split every two characters                                                                           
        if augs_manual is None:
            augmentations_use = self.params['augmentations']
        if augs_manual is not None:
            augmentations_use = augs_manual

        augs = [augmentations_use[i:i+2] for i in range(0, len(augmentations_use), 2)]

        for aug in augs:
            if aug not in self.aug_to_func.keys():
                sys.exit(f"Augmentation abbreviation {aug} is not an available key. Choose from", self.aug_to_name.key())

            self.aug_to_func[aug]() # () required to actually call function                                    

        if self.params['verbose']:
            print(f"\nUsing augmentations \n{self.augmentations_names}\n {self.augmentations}")

        return self.augmentations, self.augmentations_names


Overwriting ../cloud_seg/utils/augmentations.py


In [24]:
train_transforms_names = ['a', 'b', 'c']
string

'a_b_c'

### Model training script

In [28]:
%%file ../scripts/train_unet.py

import shutil
import numpy as np
import pandas as pd
import pandas_path as path
from PIL import Image
import torch
import pytorch_lightning as pl
import glob
from pathlib import Path
import argparse
import os

from pytorch_lightning import loggers as pl_loggers
import albumentations as A

from cloud_seg.models.unet.cloud_model import CloudModel
from cloud_seg.utils.augmentations import CloudAugmentations

DATA_DIR = Path.cwd().parent.resolve() / "data/"
DATA_DIR_MODEL_TRAINING = DATA_DIR / "model_training/"
DATA_DIR_CLOUDLESS = DATA_DIR / 'cloudless/tif/'
DATA_DIR_CLOUDS = DATA_DIR / 'clouds/'

TRAIN_FEATURES = DATA_DIR / "train_features"
TRAIN_FEATURES_NEW = DATA_DIR / "train_features_new"

TRAIN_LABELS = DATA_DIR / "train_labels"

band_mean_std = np.load(DATA_DIR / 'measured_band_stats.npy', allow_pickle=True).item()

def main(args):
    
    hparams = vars(args)
    if hparams['verbose']: print("Parameters are: ", hparams)

    pl.seed_everything(hparams['seed'], workers=True)

    hparams['bands_use'] = sorted(hparams['bands'] + hparams['bands_new']) if hparams['bands_new'] is not None else hparams['bands']
    hparams['precision'] = 32
    
    hparams['band_means'] = [band_mean_std[i]['mean'] for i in hparams['bands_use']]
    hparams['band_stds'] = [band_mean_std[i]['std'] for i in hparams['bands_use']]
    hparams['OUTPUT_DIR'] = os.path.join(hparams['OUTPUT_DIR'], hparams['segmentation_model'])
    Path(hparams['OUTPUT_DIR']).mkdir(parents=True, exist_ok=True)

    
    # Set up transforms using Albumentations library
    Augs = CloudAugmentations(hparams)
    train_transforms, train_transforms_names = Augs.add_augmentations()
    train_transforms = A.Compose(train_transforms)

    augs_val = ''
    val_transforms, val_transforms_names = Augs.add_augmentations(augs_val)
    val_transforms = A.Compose(val_transforms)

    print(train_transforms_names, val_transforms_names)
    print(train_transforms, val_transforms)
    
    
    # set up logger and model outputs to have meaningful name
    augmentations_used_string = '_'.join([name for name in train_transforms_names]) 

    dataset_str = 'originaldata'
    if hparams['cloud_augment']:
        dataset_str += '_cloudaugment'
    
    hparams['model_training_name'] = f"{len(hparams['bands_use'])}band_{dataset_str}_{hparams['encoder_name']}_{hparams['loss_function']}_{augmentations_used_string}"
    if hparams['test_run']:
        model_training_name = 'test'
    
    hparams['LOG_DIR'] = os.path.join(hparams['OUTPUT_DIR'], hparams['model_training_name'], hparams['LOG_DIR'])
    hparams['MODEL_DIR'] = os.path.join(hparams['OUTPUT_DIR'], hparams['model_training_name'], hparams['MODEL_DIR'])
                                      
    Path(hparams['LOG_DIR']).mkdir(parents=True, exist_ok=True)
    Path(hparams['MODEL_DIR']).mkdir(parents=True, exist_ok=True)
    

    # Load Data
    val_x = pd.read_csv(DATA_DIR_MODEL_TRAINING / f"validate_features_meta_cv{hparams['cross_validation_split']}.csv")
    val_y = pd.read_csv(DATA_DIR_MODEL_TRAINING / f"validate_labels_meta_cv{hparams['cross_validation_split']}.csv")
    
    # shuffle validation, such that each batch will have samples from different locations,
    # as validation_dataloader has shuffle=False
    val_x = val_x.sample(frac=1, random_state=42).reset_index(drop=True)
    val_y = val_y.sample(frac=1, random_state=42).reset_index(drop=True)
    
    if hparams['verbose']: print(val_y.head())
    
    train_x = pd.read_csv(DATA_DIR_MODEL_TRAINING / f"train_features_meta_cv{hparams['cross_validation_split']}.csv")
    train_y = pd.read_csv(DATA_DIR_MODEL_TRAINING / f"train_labels_meta_cv{hparams['cross_validation_split']}.csv")

    if not hparams['cloud_augment']:
        
        df_cloudbank = None
    
    if hparams['cloud_augment']:

        train_x_cloudless = pd.read_csv(DATA_DIR_MODEL_TRAINING / f"train_features_cloudless_meta_cv{hparams['cross_validation_split']}.csv", index=False)
        train_y_cloudless = pd.read_csv(DATA_DIR_MODEL_TRAINING / f"train_labels_cloudless_meta_cv{hparams['cross_validation_split']}.csv", index=False)

        train_y = train_y.append(train_y_cloudless, ignore_index=True)
        train_x = train_x.append(train_x_cloudless, ignore_index=True)

        df_cloudbank = pd.read_csv(DATA_DIR_MODEL_TRAINING / f"cloudbank_meta_cv{hparams['cross_validation_split']}.csv")

    if hparams['test_run']:
        nuse = hparams['test_run_nchips']

        train_x = train_x.iloc[:nuse]
        train_y = train_y.iloc[:nuse]

        val_x = val_x.iloc[:nuse]
        val_x = val_x.iloc[:nuse]

        df_cloudbank = df_cloudbank.iloc[:nuse] if df_cloudbank is not None else None
        

    # Set up models and callbacks
    cloud_model = CloudModel(
        bands=hparams['bands_use'],
        x_train=train_x,
        y_train=train_y,
        x_val=val_x,
        y_val=val_y,
        cloudbank=df_cloudbank,
        train_transforms=train_transforms,
        val_transforms=val_transforms,
        hparams=hparams,
    )

    tb_logger = pl_loggers.TensorBoardLogger(
        save_dir=hparams['LOG_DIR'],
        name='log',
    )

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        dirpath=hparams['MODEL_DIR'],
        filename='{epoch}-{val_iou_epoch:.2f}',
        monitor="val_iou_epoch",
        mode="max",
        verbose=True,
        save_last=True
    )

    early_stopping_callback = pl.callbacks.early_stopping.EarlyStopping(
        monitor="val_iou_epoch",
        patience=(cloud_model.patience * 3),
        mode="max",
        verbose=True,
    )

    lr_monitor = pl.callbacks.LearningRateMonitor(
        logging_interval='epoch'
    )

    # Train model
    # "ddp_spawn" needed for interactive jupyter, but best to use "ddp" if not
    trainer = pl.Trainer(
        gpus=-1,
        # deterministic=True,
        fast_dev_run=False,
        # profiler="simple",
        # max_epochs=2,
        # overfit_batches=1,
        # auto_scale_batch_size=True,
        check_val_every_n_epoch=1,
        num_sanity_val_steps=2,
        precision=hparams['precision'],
        strategy=hparams['strategy'],
        # plugins=DDPSpawnPlugin(find_unused_parameters=False),
        callbacks=[checkpoint_callback, early_stopping_callback, lr_monitor],
        logger=tb_logger,
    )

    # Fit the model
    trainer.fit(model=cloud_model)
    
if __name__=='__main__':
    
    parser = argparse.ArgumentParser(description='runtime parameters')
    
    # Data and IO
    parser.add_argument("--bands", nargs='+' , default=["B02", "B03", "B04", "B08"],
                        help="bands desired")
    
    parser.add_argument("--bands_new", nargs='+', default=None,
                        help="additional bands to use beyond original four")
    
    parser.add_argument("-cv", "--cross_validation_split", type=int, default=0,
                        help="cross validation split to use for training") 

    parser.add_argument("--OUTPUT_DIR", type=str, default='../trained_models/',
                        help="Directory to save logs and trained models model")
                                      
    parser.add_argument("--LOG_DIR", type=str, default='logs/',
                        help="Sub-directory of OUTPUT_DIR to save logs")
    
    parser.add_argument("--MODEL_DIR", type=str, default='model/',
                        help="Sub-directory of OUTPUT_DIR to save logs")

    parser.add_argument("--seed", type=int , default=13579,
                        help="random seed for train test split")
   
    parser.add_argument("-v", "--verbose", action="store_true",
                        help="increase output verbosity")
   


    # Training (gpus, optimization, etc...)
    parser.add_argument("--gpu", action="store_true",
                        help="Use GPU")
    
    parser.add_argument("--strategy", type=str, default='ddp',
                        help="Distributed training strategy")
        
    parser.add_argument("--test_run", action="store_true",
                        help="Subsample training and validation data")
    
    parser.add_argument("--test_run_nchips", type=int, default=512,
                        help="Subsample training and validation data to this size")

    parser.add_argument("--num_workers", type=int, default=3,
                        help="number of data loader workers")
    
    parser.add_argument("--persistent_workers", action="store_false",
                        help="Persistent data loader workers")
    
    parser.add_argument("--batch_size", type=int, default=8,
                        help="Batch size for model training")
    
    parser.add_argument("--loss_function", type=str, default='dice',
                        help="loss_function to use", choices=['bce', 'Dice'])
      
    parser.add_argument("--learning_rate", type=float, default=1e-3,
                        help="Learning rate for model optimization")
  
    parser.add_argument("--optimizer", type=str, default='ADAM',
                        help="Optimizer to use", choices=['ADAM', 'SGD'])
    
    parser.add_argument("--scheduler", type=str, default='plateau',
                        help="Learning rate scheduler to use", choices=['plateau', 'EXPONENTIAL', 'cosine'])
    
    parser.add_argument("--plot_validation_images", action="store_true",
                        help="Plot final batch to tensorboard")
              
        
    # Models and Augmentations
    parser.add_argument("--segmentation_model", type=str, default='unet',
                        help="Encocoder architecture to use", choices=['unet', 'DeepLabV3Plus'])
  
    parser.add_argument("--encoder_name", type=str, default='efficientnet-b0',
                        help="Encocoder architecture to use", choices=['efficientnet-b0', 'resnet18', 'resnet34', 'vgg19_bn'])
  
    parser.add_argument("--augmentations", type=str, default='vfhfrrnr',
                        help="training augmentations to use")
    
    parser.add_argument("--cloud_augment", action="store_true",
                        help="Use cloud augmentation")

    args = parser.parse_args()
    
    main(args)

Overwriting ../scripts/train_unet.py


# Model predict script

In [4]:
%%file ../scripts/predict_unet.py

import shutil
import numpy as np
import pandas as pd
import pandas_path as path
from PIL import Image
import torch
import pytorch_lightning as pl
import glob
from pathlib import Path

from typing import List
# import typer
import logging

import os
import argparse

from cloud_seg.models.unet.cloud_model import CloudModel
from cloud_seg.models.unet.cloud_model import CloudDataset


parser = argparse.ArgumentParser(description='runtime parameters')
parser.add_argument("--bands", nargs='+' , default=["B02", "B03", "B04", "B08"],
                    help="bands desired")
parser.add_argument("--bands_new", nargs='+', default=None,
                    help="additional bands to use beyond original four")

parser.add_argument("-cv", "--cross_validation_split", type=int, default=0,
                    help="cross validation split to use for training") 

parser.add_argument("--batch_size", type=int, default=8,
                    help="Batch size for model inference")

parser.add_argument("--INPUT_DIR", type=str, default='../trained_models/unet/4band_originaldata_efficientnet-b0_dice__Normalize_VerticalFlip_HorizontalFlip_RandomRotate90/',
                    help="Directory to save logs and trained models model")

parser.add_argument("--LOG_DIR", type=str, default='logs/',
                    help="Sub-directory of OUTPUT_DIR to save logs")

parser.add_argument("--MODEL_DIR", type=str, default='model/',
                    help="Sub-directory of OUTPUT_DIR to save logs")

parser.add_argument("--OUTPUT_DIR", type=str, default='predictions/',
                    help="Directory to save logs and trained models model")

parser.add_argument("--gpu", action="store_true",
                    help="Use GPU")  
                    
parser.add_argument("-v", "--verbose", action="store_true",
                    help="increase output verbosity")
                    
parser.add_argument("--local_run", action="store_true",
                    help="Whether running locally or on planetary computer")
                    
parser.add_argument("--model_name", type=str, default='cloud_model.pt',
                    help="directory to save trained model")

parser.add_argument("--segmentation_model", type=str, default='unet',
                    help="Encocoder architecture to use", choices=['unet', 'DeepLabV3Plus'])
  
parser.add_argument("--encoder_name", type=str, default='efficientnet-b0',
                    help="Architecture to use", choices=['efficientnet-b0', 'resnet34'])

parser.add_argument("--load_checkpoint", action="store_true",
                    help="Whether loading weights from checkpoint (.ckpt) or just from saved weights state_dict (.pt)")

hparams = vars(parser.parse_args())
hparams['weights'] = None
hparams['bands_use'] = sorted(hparams['bands'] + hparams['bands_new']) if hparams['bands_new'] is not None else hparams['bands']
         
# hparams['INPUT_DIR'] = os.path.join(hparams['INPUT_DIR'], hparams['segmentation_model'], hparams['model_training_name'])
# hparams['MODEL_DIR'] = os.path.join(hparams['INPUT_DIR'], hparams['model_training_name'], hparams['MODEL_DIR'])
# hparams['LOG_DIR'] = os.path.join(hparams['INPUT_DIR'], hparams['model_training_name'], hparams['OUTPUT_DIR'], hparams['LOG_DIR'])
# hparams['OUTPUT_DIR'] = os.path.join(hparams['INPUT_DIR'], hparams['model_training_name'], hparams['OUTPUT_DIR'])

# Path(hparams['LOG_DIR']).mkdir(parents=True, exist_ok=True)
# Path(hparams['OUTPUT_DIR']).mkdir(parents=True, exist_ok=True)

if hparams['local_run']:      
    
    ROOT_DIR = Path.cwd().parent.resolve()
    ASSETS_DIR = Path(hparams['INPUT_DIR'])
    MODEL_PATH = ASSETS_DIR / hparams['MODEL_DIR'] / "last.ckpt"
                    
    PREDICTIONS_DIR = ASSETS_DIR / hparams['OUTPUT_DIR']
     
    DATA_DIR = ROOT_DIR / "data/"
    DATA_DIR_MODEL_TRAINING = DATA_DIR / "model_training/"
    DATA_DIR_CLOUDLESS = DATA_DIR / 'cloudless/tif/'
    DATA_DIR_CLOUDS = DATA_DIR / 'clouds/'

    INPUT_IMAGES_DIR = DATA_DIR / "train_features"
    INPUT_IMAGES_DIR_NEW = DATA_DIR / "train_features_new"

    TRAIN_LABELS = DATA_DIR / "train_labels"

    band_mean_std = np.load(DATA_DIR / 'measured_band_stats.npy', allow_pickle=True).item()
       
    logger = logging.getLogger("test_logger")
    fh = logging.FileHandler('test_logger.log')
    ch = logging.StreamHandler()
    
    logger.addHandler(fh)
    logger.addHandler(ch)

    Path(PREDICTIONS_DIR).mkdir(parents=True, exist_ok=True)
  
else:
    ROOT_DIR = Path("/codeexecution")
    PREDICTIONS_DIR = ROOT_DIR / "predictions"
    ASSETS_DIR = ROOT_DIR / "assets"
                    
    DATA_DIR = ROOT_DIR / "data"
    INPUT_IMAGES_DIR = DATA_DIR / "test_features"
                    
    # Set the pytorch cache directory and include cached models in your submission.zip
    os.environ["TORCH_HOME"] = str(ASSETS_DIRECTORY / "assets/torch")

    MODEL_PATH = ASSETS_DIR / hparams['model_name']            

logger = logging.getLogger()

def get_metadata(features_dir: os.PathLike, bands: List[str]):
    """
    Given a folder of feature data, return a dataframe where the index is the chip id
    and there is a column for the path to each band's TIF image.

    Args:
        features_dir (os.PathLike): path to the directory of feature data, which should have
            a folder for each chip
        bands (list[str]): list of bands provided for each chip
    """
    chip_metadata = pd.DataFrame(index=[f"{band}_path" for band in bands])
    chip_ids = (
        pth.name for pth in features_dir.iterdir() if not pth.name.startswith(".")
    )

    for chip_id in sorted(chip_ids):
        chip_bands = [features_dir / chip_id / f"{band}.tif" for band in bands]
        chip_metadata[chip_id] = chip_bands

    return chip_metadata.transpose().reset_index().rename(columns={"index": "chip_id"})


def make_predictions(
    model: CloudModel,
    x_paths: pd.DataFrame,
    bands: List[str],
    predictions_dir: os.PathLike,
):
    """Predicts cloud cover and saves results to the predictions directory.

    Args:
        model (CloudModel): an instantiated CloudModel based on pl.LightningModule
        x_paths (pd.DataFrame): a dataframe with a row for each chip. There must be a column for chip_id,
                and a column with the path to the TIF for each of bands provided
        bands (list[str]): list of bands provided for each chip
        predictions_dir (os.PathLike): Destination directory to save the predicted TIF masks
    """
    predict_dataset = CloudDataset(
        x_paths=x_paths,
        bands=bands,
    )
    predict_dataloader = torch.utils.data.DataLoader(
        predict_dataset,
        batch_size=model.batch_size,
        num_workers=model.num_workers,
        shuffle=False,
        pin_memory=True,
    )
    
    for batch_index, batch in enumerate(predict_dataloader):
        print("Running on batch: ", batch_index)
        logger.debug(f"Predicting batch {batch_index} of {len(predict_dataloader)}")
       
        x = batch["chip"]
        if model.gpu:
            x = x.cuda(non_blocking=True)
        
        preds = model.forward(x)
        if not hparams['local_run']:
            preds = (preds > 0.5)
            
        preds = preds.detach()
        
        if model.gpu:
            preds = preds.to("cpu").numpy()
        if not hparams['local_run']:
            preds = preds.astype("uint8")

        for chip_id, pred in zip(batch["chip_id"], preds):
            chip_pred_path = predictions_dir / f"{chip_id}.tif"
            chip_pred_im = Image.fromarray(pred)
            chip_pred_im.save(chip_pred_path)


def main(
    bands: List[str] = ["B02", "B03", "B04", "B08"],
    fast_dev_run: bool = False,
):
    """
    Generate predictions for the chips in features_dir using the model saved at
    model_path.

    Predictions are saved in predictions_dir. The default paths to all three files are based on
    the structure of the code execution runtime.

    Args:
        model_weights_path (os.PathLike): Path to the weights of a trained CloudModel.
        features_dir (os.PathLike, optional): Path to the features for the data. Defaults
            to 'data/test_features' in the same directory as main.py
        predictions_dir (os.PathLike, optional): Destination directory to save the predicted TIF masks
            Defaults to 'predictions' in the same directory as main.py
        bands (List[str], optional): List of bands provided for each chip
    """
    if not INPUT_IMAGES_DIR.exists():
        raise ValueError(
            f"The directory for feature images must exist and {INPUT_IMAGES_DIR} does not exist"
        )
    PREDICTIONS_DIR.mkdir(exist_ok=True, parents=True)

    print('RUNNING')
    logger.info("Loading model")
    print('RUNNING')
    
    # Load with gpu=False, then put on GPU
    hparams['gpu'] = False
    model = CloudModel(
        bands=hparams['bands_use'],
        hparams=hparams
    )
   
    print('Constructed base model')
    # load model from disk
    if not hparams['load_checkpoint']:
        # directly load weights
        model.load_state_dict(torch.load(MODEL_PATH))
        
    if hparams['load_checkpoint']:
        # load weights from checkpoint
        checkpoint = torch.load(MODEL_PATH)
        model.load_state_dict(checkpoint['state_dict'])
    print('Loaded model weights')
    
    hparams['gpu'] = True
    if hparams['gpu']:
        model = model.cuda()
        model.gpu = True

             
    # Load metadata
    logger.info("Loading metadata")
    metadata = get_metadata(INPUT_IMAGES_DIR, bands=bands)
    if fast_dev_run:
        metadata = metadata.head()
    logger.info(f"Found {len(metadata)} chips")
    
    
    print('Loaded metadata')
    # Make predictions and save to disk
    logger.info("Generating predictions in batches")
    make_predictions(model, metadata, bands, PREDICTIONS_DIR)

    logger.info(f"""Saved {len(list(PREDICTIONS_DIR.glob("*.tif")))} predictions""")


if __name__ == "__main__":
    # if hparams['local_run']:              
    #     main()
    # else:
        #typer.run(main)
    main()

Overwriting ../scripts/predict_unet.py


In [50]:
import torch

MODEL_PATH = '../../trained_models/unet/test/epoch=21-val_iou_epoch=0.84.ckpt'
checkpoint = torch.load(MODEL_PATH, map_location=torch.device('cpu'))
for k, v in checkpoint.items():
    print(k)
# model.load_state_dict(checkpoint['model'])

epoch
global_step
pytorch-lightning_version
state_dict
callbacks
optimizer_states
lr_schedulers
hparams_name
hyper_parameters
kwargs


In [5]:
%%file {unet_model_dir}/cloud_dataset.py

import numpy as np
import pandas as pd
import rasterio
import torch
from typing import Optional, List
import torchvision

class CloudDataset(torch.utils.data.Dataset):
    """Reads in images, transforms pixel values, and serves a
    dictionary containing chip ids, image tensors, and
    label masks (where available).
    """

    def __init__(
        self,
        x_paths: pd.DataFrame,
        bands: List[str],
        y_paths: Optional[pd.DataFrame] = None,
        cloudbank: Optional[pd.DataFrame] = None,
        transforms: Optional[list] = None,
    ):
        """
        Instantiate the CloudDataset class.

        Args:
            x_paths (pd.DataFrame): a dataframe with a row for each chip. There must be a column for chip_id,
                and a column with the path to the TIF for each of bands
            bands (list[str]): list of the bands included in the data
            y_paths (pd.DataFrame, optional): a dataframe with a row for each chip and columns for chip_id
                and the path to the label TIF with ground truth cloud cover
            cloudbank (pd.DataFrame, optional): a dataframe with a row for each cloud chip, columns for chip_id
                and the path to the cloud band TIFs and label TIF with ground truth cloud cover.
            transforms (list, optional): list of transforms to apply to the feature data (eg augmentations)
        """
        self.data  = x_paths
        self.label = y_paths
        self.cloudbank = cloudbank
        if cloudbank is not None:
            self.len_cloudbank = len(cloudbank)
        
        self.transforms = transforms
        self.bands = bands
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        # Loads an n-channel image from a chip-level dataframe
        img = self.data.loc[idx]
        band_arrs = []
        for band in self.bands:
            with rasterio.open(img[f"{band}_path"]) as b:
                band_arr = b.read(1).astype("float32")
            band_arrs.append(band_arr)
            
        x_arr = np.stack(band_arrs, axis=-1) # images in (B, H, W, C)

        # Load label if available
        if self.label is not None:
            label_path = self.label.loc[idx].label_path
            if label_path != 'None':
                with rasterio.open(label_path) as lp:
                    y_arr = lp.read(1).astype("float32")
            else:
                # This is a cloudless image, so sample a random cloud chip from cloudbank
                # load in new cloud label, and add cloud band data to x_arr bands
                idx_cloud = np.random.randint(0, self.len_cloudbank)
                cloud_paths = self.cloudbank.loc[idx_cloud]
                
                # load label
                with rasterio.open(cloud_paths.label_path) as lp:
                    y_arr = lp.read(1).astype("float32")  
                    
                # load cloud bands
                band_arrs = []
                for band in self.bands:
                    with rasterio.open(cloud_paths[f"{band}_path"]) as b:
                        band_arr = b.read(1).astype("float32")
                    band_arrs.append(band_arr)
                    
                # add clouds to cloudless image
                x_arr += np.stack(band_arrs, axis=-1)

        # Prepare dictionary for item
        item = {}
        item["chip_id"] = img.chip_id
        
        # Apply data augmentations, if provided
        if self.label is not None:
            # Apply same data augmentations to the label
            if self.transforms:
                transformed = self.transforms(image=x_arr, mask=y_arr)
                x_arr = transformed["image"]
                y_arr = transformed["mask"]
                
            item["label"] = y_arr
        if self.label is None:
            if self.transforms:
                x_arr = self.transforms(image=x_arr)["image"]
                
        x_arr = np.transpose(x_arr, [2, 0, 1]) # put images in (B, C, H, W)

        item["chip"] = x_arr
        
        return item

Overwriting ../cloud_seg/models/unet/cloud_dataset.py


In [6]:
%%file {unet_model_dir}/losses.py
import numpy as np
import torch

from typing import Sequence, Optional, Union

def intersection_and_union(pred, true):
    """
    Calculates intersection and union for a batch of images.

    Args:
        pred (torch.Tensor): a tensor of predictions
        true (torc.Tensor): a tensor of labels

    Returns:
        intersection (int): total intersection of pixels
        union (int): total union of pixels
    """
    # valid_pixel_mask = true.ne(255)  # valid pixel mask
    # true = true.masked_select(valid_pixel_mask).to("cpu")
    # pred = pred.masked_select(valid_pixel_mask).to("cpu")

    # Intersection and union totals
    pred_flattened = pred.view(-1)
    true_flattened = true.view(-1)

    intersection = torch.logical_and(true_flattened, pred_flattened)
    union = torch.logical_or(true_flattened, pred_flattened)
    
    return torch.sum(intersection).float(), torch.sum(union).float()#, torch.sum(intersection) / torch.sum(union)

def dice_loss(pred, true, dice_smooth=1.):
    
    pred_flattened = pred.view(-1)
    true_flattened = true.view(-1)

    intersection = (pred_flattened * true_flattened).sum()
    
    return 1 - ((2. * intersection + dice_smooth) /
              (pred_flattened.sum() + true_flattened.sum() + dice_smooth))

class DiceLoss(torch.nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
                
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice


Overwriting ../cloud_seg/models/unet/losses.py


In [7]:
%%file {unet_model_dir}/metrics.py

import numpy as np
import torch
import torchmetrics

class Intersection(torchmetrics.Metric):
    def __init__(self, dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)

        self.add_state("intersection", default=torch.tensor(0), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        preds, target = self._input_format(preds, target)
        assert preds.shape == target.shape

        self.intersection += torch.logical_and(preds.view(-1), target.view(-1))
        # self.correct += torch.sum(preds == target)
        # self.total += target.numel()

    def compute(self):
        return self.intersection.float()
        # return self.correct.float() / self.total

Overwriting ../cloud_seg/models/unet/metrics.py


In [8]:
%%file {unet_model_dir}/callbacks.py

# Adapted from https://github.com/PyTorchLightning/Lightning-Bolts/blob/master/pl_bolts/callbacks/vision/confused_logit.py#L20-L167
from typing import Sequence

import torch
from pytorch_lightning import Callback, LightningModule, Trainer
from torch import Tensor, nn

from matplotlib import pyplot as plt
from matplotlib.axes import Axes
from matplotlib.figure import Figure

# from pytorch_lightning.utilities import rank_zero_only
# @rank_zero_only
class DisplayChipsCallback(Callback):  # pragma: no cover
    """Takes the input chip, true label, and label prediction
        trainer = Trainer(callbacks=[DisplayChips()])
    .. note:: Whenever called, this model will look for ``self.last_batch`` and ``self.last_logits``
              in the LightningModule.
    """

    def __init__(
        self,
        num_images_plot: int=4,
    ):
        """
        Args:
            top_k: How many  images we should plot
   
        """
        super().__init__()
        self.num_images_plot = num_images_plot

    def on_validation_epoch_end(
        self,
        trainer: Trainer,
        pl_module: LightningModule,
        # outputs: Sequence,
        # batch: Sequence,
        # batch_idx: int,
        # dataloader_idx: int,
    ) -> None:
        # show images only every 20 batches
        # if batch_idx != 0:
        #     return

        # pick the last batch and logits
        # x, y = batch["chip"], batch["label"]
        try:
            x = pl_module.last_x.to("cpu")
            y = pl_module.last_y.to("cpu")
            pred = pl_module.last_pred.to("cpu")
            
        except AttributeError as err:
            m = """please track the last_pred in the validation_step like so:
                def validation_step(...):
                    self.last_pred = your_pred
            """
            raise AttributeError(m) from err

        print(pred)
        self._plot(x, y, pred, trainer, pl_module)

    def _plot(
        self,
        x: Tensor,
        y: Tensor,
        pred: Tensor,
        trainer: Trainer,
        model: LightningModule,
    ) -> None:

        batch_size, c, w, h = x.size()

        # final batch may not be full size
        nimg_plt = self.min(batch_size, self.num_images_plot)
        
        fig, axarr = plt.subplots(nrows=nimg_plt, ncols=3, figsize=(15, 5*))
       
        for img_i in range(nimg_plt):
            xi = x[img_i].to("cpu")
            yi = y[img_i].to("cpu")
            predi = pred[img_i].to("cpu")
            
            self.__draw_data_sample(fig, axarr, img_i, 0, xi[0], "Chip")
            self.__draw_label_sample(fig, axarr, img_i, 1, yi, "True label")
            self.__draw_label_sample(fig, axarr, img_i, 2, predi, "Prediction")
            
        # model.logger.experiment.add_figure("validation_predictions", fig, global_step=trainer.global_step)
        # trainer.logger.experiment[0].add_image("validation_predictions", fig, global_step=trainer.global_step)
        # model.log("validation_predictions", fig, global_step=trainer.global_step)

    @staticmethod
    def __draw_data_sample(fig: Figure, axarr: Axes, row_idx: int, col_idx: int, img: Tensor, title: str) -> None:
        im = axarr[row_idx, col_idx].imshow(img)
        axarr[row_idx, col_idx].set_title(title, fontsize=20)
        
    @staticmethod
    def __draw_label_sample(fig: Figure, axarr: Axes, row_idx: int, col_idx: int, img: Tensor, title: str) -> None:
        im = axarr[row_idx, col_idx].imshow(img, vmin=0., vmax=1.)
        axarr[row_idx, col_idx].set_title(title, fontsize=20)
        
                

Overwriting ../cloud_seg/models/unet/callbacks.py


In [9]:
%%file {unet_model_dir}/plotting_tools.py

import numpy as np
import torch
from torch import Tensor

from matplotlib import pyplot as plt
import xarray
import xrspatial.multispectral as ms

# from pytorch_lightning.utilities import rank_zero_only
# @rank_zero_only

def to_xarray(im_arr):
    """Put images in xarray.DataArray format"""

    return xarray.DataArray(im_arr, dims=["y", "x"])

def true_color_img(img, normalized=True):
    """Given the path to the directory of Sentinel-2 chip feature images,
    plots the true color image"""
    
    band_mean_std = {'B02': {'mean': 2848.064112016446,
    'std': 3156.9268464765087,
    'min': 0,
    'max': 27600},
    'B03': {'mean': 2839.0871485290295,
    'std': 2899.280144509762,
    'min': 0,
    'max': 26096},
    'B04': {'mean': 2741.2891076425326,
    'std': 2789.961608891907,
    'min': 0,
    'max': 23104},
    'B08': {'mean': 3657.9092112857143,
    'std': 2424.18942846055,
    'min': 0,
    'max': 19568}}

    if normalized:
        img[2] = img[2]*band_mean_std['B04']['std'] + band_mean_std['B04']['mean']
        img[1] = img[1]*band_mean_std['B03']['std'] + band_mean_std['B03']['mean']
        img[0] = img[0]*band_mean_std['B02']['std'] + band_mean_std['B02']['mean']
        
    red = to_xarray(img[2])
    green = to_xarray(img[1])
    blue = to_xarray(img[0])
    
    return ms.true_color(r=red, g=green, b=blue)

def plot_prediction_grid(x: Tensor, y: Tensor, pred: Tensor, chip_id, num_images_plot: int = 4, fontsize=18):

        batch_size, c, w, h = x.size()
        
        nimg_plt = min(batch_size, num_images_plot)

        fig, axarr = plt.subplots(nrows=nimg_plt, ncols=3, figsize=(15, 5*nimg_plt))
       
        for img_i in range(nimg_plt):
            
            chip_idi = chip_id[img_i]
            xi = true_color_img(x[img_i].to("cpu").numpy().astype(np.float32), normalized=True)
            yi = y[img_i].to("cpu")
            predi = pred[img_i].to("cpu")
            
            axarr[img_i, 0].imshow(xi)
            axarr[img_i, 0].set_title(f"{chip_idi}", fontsize=fontsize)
            
            axarr[img_i, 1].imshow(yi, vmin=0., vmax=1.)
            axarr[img_i, 1].set_title("True label", fontsize=fontsize)
            
            axarr[img_i, 2].imshow(predi, vmin=0., vmax=1.)
            axarr[img_i, 2].set_title("Prediction", fontsize=fontsize)
            
        plt.close(fig)
        
        return fig
                

Overwriting ../cloud_seg/models/unet/plotting_tools.py


In [28]:
%%file {unet_model_dir}/cloud_model.py

from typing import Optional, List

import pandas as pd
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import torch
import torchmetrics
# from pytorch_lightning.utilities import rank_zero_only
# from pytorch_lightning.loggers.base import rank_zero_experiment

from .cloud_dataset import CloudDataset
from .losses import intersection_and_union
from .losses import dice_loss
from .plotting_tools import plot_prediction_grid


class CloudModel(pl.LightningModule):
    def __init__(
        self,
        bands: List[str],
        x_train: Optional[pd.DataFrame] = None,
        y_train: Optional[pd.DataFrame] = None,
        x_val: Optional[pd.DataFrame] = None,
        y_val: Optional[pd.DataFrame] = None,
        cloudbank: Optional[pd.DataFrame] = None,
        train_transforms = None,
        val_transforms = None,
        hparams: dict = {},
    ):
        """
        Instantiate the CloudModel class based on the pl.LightningModule
        (https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html).

        Args:
            bands (list[str]): Names of the bands provided for each chip
            x_train (pd.DataFrame, optional): a dataframe of the training features with a row for each chip.
                There must be a column for chip_id, and a column with the path to the TIF for each of bands.
                Required for model training
            y_train (pd.DataFrame, optional): a dataframe of the training labels with a for each chip
                and columns for chip_id and the path to the label TIF with ground truth cloud cover.
                Required for model training
            x_val (pd.DataFrame, optional): a dataframe of the validation features with a row for each chip.
                There must be a column for chip_id, and a column with the path to the TIF for each of bands.
                Required for model training
            y_val (pd.DataFrame, optional): a dataframe of the validation labels with a for each chip
                and columns for chip_id and the path to the label TIF with ground truth cloud cover.
                Required for model training
            cloudbank (pd.DataFrame, optional): a dataframe of paths to additional clouds to sample from. 
                Optional for model training, but required if using chips where label_path=='None'
            hparams (dict, optional): Dictionary of additional modeling parameters.
        """
        super().__init__()
        self.hparams.update(hparams)
        self.save_hyperparameters()

        # required
        self.bands = bands
        self.num_channels = len(bands)
        
        # optional modeling params
        self.segmentation_model = self.hparams.get("segmentation_model", "unet")
        self.encoder_name = self.hparams.get("encoder_name", "efficientnet-b0")
        self.weights = self.hparams.get("weights", None)
        
        self.loss_function = self.hparams.get("loss_function", "dice")        
        self.optimizer = self.hparams.get("optimizer", "ADAM")
        self.scheduler = self.hparams.get("scheduler", "PLATEAU")
        
        self.learning_rate = self.hparams.get("learning_rate", 1e-3)
        self.momentum = self.hparams.get("momentum", 0.9)
        self.T_0 = self.hparams.get("T_0", 10)
        self.eta_min = self.hparams.get("eta_min", 1e-5)
      
        self.reduce_learning_rate_factor = self.hparams.get("reduce_learning_rate_factor", 0.1)

        self.patience = self.hparams.get("patience", 5)
        self.learning_rate_patience = self.hparams.get("learning_rate_patience", 5)
        self.batch_size = self.hparams.get("batch_size", 8)

        self.num_workers = self.hparams.get("num_workers", 2)
        self.pin_memory = self.hparams.get("pin_memory", True)
        self.persistent_workers = self.hparams.get("persistent_workers", False)
        
        self.gpu = self.hparams.get("gpu", False)
        
        self.log_on_step = self.hparams.get("log_on_step", False)
        self.progress_bar = self.hparams.get("progress_bar", False)
        
        self.plot_validation_images = self.hparams.get("plot_validation_images", True)
        self.num_images_plot = self.hparams.get("num_images_plot", self.batch_size)

        self.train_transform = train_transforms
        self.val_transform = val_transforms

        # Instantiate datasets, model, and trainer params if provided
        self.train_dataset = CloudDataset(
            x_paths=x_train,
            bands=self.bands,
            y_paths=y_train,
            cloudbank=cloudbank,
            transforms=self.train_transform,
        )
        self.val_dataset = CloudDataset(
            x_paths=x_val,
            bands=self.bands,
            y_paths=y_val,
            transforms=self.val_transform,
        )
        
        # define some performance metrics using torchmetrics
        # self.train_accuracy = torchmetrics.Accuracy()
        # self.val_intersection = mymetrics.Intersection()
        self.val_IoU = torchmetrics.IoU(num_classes=2)
        self.train_IoU = torchmetrics.IoU(num_classes=2)

        self.model = self._prepare_model()

    ## Required LightningModule methods ##
    def forward(self, image: torch.Tensor):
        # Forward pass
        # output of model is (B, 1, H, W), so remove axis=1
        if self.loss_function == "BCE":
            # return raw logits in order to use BCEWithLogitsLoss
            # which is more stable than BCE:
            # https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html#torch.nn.BCEWithLogitsLoss
            return self.model(image).view(-1, 512, 512)
        
        else:
            return torch.sigmoid(self.model(image).view(-1, 512, 512))
            

    def calculate_loss(self, chip, label, preds):
        if self.loss_function.upper()=="BCE":
            loss = torch.nn.BCEWithLogitsLoss(reduction="none")(preds, label.float()).mean()
            
        if self.loss_function.upper()=="DICE":
            loss = dice_loss(preds, label)

            # loss = DiceLoss()(preds, label)
            # loss = smp.losses.JaccardLoss()(preds, label)
        #if self.loss_function.upper()=="IOU":
        #     loss = torchmetrics.IoU(num_classes=2)(preds, label)

        return loss

    def training_step(self, batch: dict, batch_idx: int):
        """
        Training step.

        Args:
            batch (dict): dictionary of items from CloudDataset of the form
                {'chip_id': list[str], 'chip': list[torch.Tensor], 'label': list[torch.Tensor]}
            batch_idx (int): batch number
        """
        if self.train_dataset.data is None:
            raise ValueError(
                "x_train and y_train must be specified when CloudModel is instantiated to run training"
            )

        # Switch on training mode
        self.model.train()
        torch.set_grad_enabled(True)

        # Load images and labels
        x = batch["chip"]
        y = batch["label"].long()
        if self.gpu:
            x, y = x.cuda(non_blocking=True), y.cuda(non_blocking=True)

        # Forward pass
        preds = self.forward(x)

        loss = self.calculate_loss(x, y, preds)

        if self.loss_function == "BCE":
            preds = torch.sigmoid(preds)
            
        
        preds = (preds > 0.5) * 1  # convert to int

        # batch_intersection, batch_union = intersection_and_union(preds, y)
    
        self.train_IoU(preds, y)

        self.log(
            "train_performance", 
            {"iou": self.train_IoU},
            on_step=self.log_on_step,
            on_epoch=True,
            prog_bar=self.progress_bar,
        )
        self.log(
            "train_loss",
            loss,
            on_step=self.log_on_step,
            on_epoch=True,
            prog_bar=self.progress_bar,
        )

        return loss

    def validation_step(self, batch: dict, batch_idx: int):
        """
        Validation step.

        Args:
            batch (dict): dictionary of items from CloudDataset of the form
                {'chip_id': list[str], 'chip': list[torch.Tensor], 'label': list[torch.Tensor]}
            batch_idx (int): batch number
        """
        if self.val_dataset.data is None:
            raise ValueError(
                "x_val and y_val must be specified when CloudModel is instantiated to run validation"
            )

        # Switch on validation mode
        self.model.eval()
        torch.set_grad_enabled(False)

        # Load images and labels
        x = batch["chip"]
        y = batch["label"].long()
        chip_id = batch["chip_id"]
        if self.gpu:
            x, y = x.cuda(non_blocking=True), y.cuda(non_blocking=True)

        preds = self.forward(x)

        loss = self.calculate_loss(x, y, preds)

        if self.loss_function == "BCE":
            preds = torch.sigmoid(preds)
            
        preds = (preds > 0.5) * 1  # convert to int

        if self.plot_validation_images:
            # keep to pass to validation_epoch_end and plot
            self.last_x = x
            self.last_y = y
            self.last_pred = preds
            self.last_chip_id = chip_id

        # Log batch IOU
        batch_intersection, batch_union = intersection_and_union(preds, y)
        self.val_IoU(preds, y)

        self.log("val_performance", 
                 {"iou": self.val_IoU},
                 on_step=self.log_on_step, on_epoch=True, prog_bar=self.progress_bar)
                 
        self.log("val_loss", loss, on_step=self.log_on_step, on_epoch=True, prog_bar=self.progress_bar)
        
        # keep seperate to use for early stopping
        self.log("val_iou", self.val_IoU, on_step=True, on_epoch=True, prog_bar=self.progress_bar)

        return {"loss": loss}#, "x": x, "y": y, "pred": preds}

#     def validation_step_end(self, batch_parts):
#         gpu_use = 0
#         # print(batch_parts['x'][gpu_use].size())
#         return {"x": batch_parts["x"][gpu_use], "y": batch_parts["y"][gpu_use], "pred": batch_parts["pred"][gpu_use]}

    # @rank_zero_only
    # @rank_zero_experiment
    
    def validation_epoch_end(self, outputs):
        # idevice = self.last_x.get_device()
        # if idevice == 0:
        # if self.global_rank==0:
        if self.plot_validation_images:
            # self.logger[0].experiment.add_figure("chip_label_prediction", 
            self.logger.experiment.add_figure("chip_label_prediction", 
                                                 plot_prediction_grid(self.last_x,
                                                                      self.last_y,
                                                                      self.last_pred,
                                                                      self.last_chip_id,
                                                                      num_images_plot=self.num_images_plot),
                                                 self.current_epoch)

        # if batch_idx == 0:
            # print(out)
            # for out in validation_step_outputs[:1]:
            #     # output from each gpu
            #     print(out)
            
    def train_dataloader(self):
        # DataLoader class for training
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.batch_size|self.hparams.batch_size,
            num_workers=self.num_workers,
            shuffle=True,
            pin_memory=self.pin_memory,
            persistent_workers=self.persistent_workers,
        )

    def val_dataloader(self):
        # DataLoader class for validation
        return torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            drop_last=self.plot_validation_images, # if plotting last batch images ensure full last batch
            pin_memory=self.pin_memory,
            persistent_workers=self.persistent_workers,
        )

    def configure_optimizers(self):
        
        if self.optimizer.upper()=="ADAM":
            optimizer = torch.optim.Adam(
                self.model.parameters(),
                lr=self.learning_rate,
            )
            
        if self.optimizer.upper()=="ADAMW":
            optimizer = torch.optim.AdamW(
                self.model.parameters(),
                lr=self.learning_rate,
            )
            # sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=10)

        if self.optimizer.upper()=="SGD":
            optimizer = torch.optim.SGD(
                self.model.parameters(),
                lr=self.learning_rate,
                momentum=self.momentum,
            )
        
        if self.scheduler.upper()=="EXPONENTIAL":
            scheduler = torch.optim.lr_scheduler.ExponentialLR(
                optimizer,
                gamma=0.95,
            )
            
        if self.scheduler.upper()=="COSINE":
            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer,
                T_0=self.T_0,
                eta_min=self.eta_min,
            ) 
  
        if self.scheduler.upper()=="PLATEAU":
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer,
                'max',
                factor=self.reduce_learning_rate_factor,
                patience=self.learning_rate_patience,
            )
            
            return {"optimizer": optimizer, 
                    "lr_scheduler": {
                        "scheduler": scheduler,
                        "monitor": "val_iou",
                    },
            }
                                       
        return [optimizer], [scheduler]
                
    ## Convenience Methods ##
    def _prepare_model(self):
        
        if self.segmentation_model.upper()=="UNET":
            # Instantiate U-Net model
            unet_model = smp.Unet(
                encoder_name=self.encoder_name,
                encoder_weights=self.weights,
                in_channels=self.num_channels,
                classes=1,
            )
            if self.gpu:
                unet_model.cuda()
                
        if self.segmentation_model.upper()=="DEEPLABV3PLUS":
            # Instantiate DeepLabV3Plus model (https://arxiv.org/abs/1802.02611v3)
            unet_model = smp.DeepLabV3Plus(
                encoder_name=self.encoder_name,
                encoder_weights=self.weights,
                in_channels=self.num_channels,
                classes=1,
            )
            if self.gpu:
                unet_model.cuda()

        return unet_model

Overwriting ../cloud_seg/models/unet/cloud_model.py


In [6]:
# save the model
submission_assets_dir = submission_dir / "assets"
submission_assets_dir.mkdir(parents=True, exist_ok=True)

model_weight_path = submission_assets_dir / "cloud_model.pt"
torch.save(cloud_model.state_dict(), model_weight_path)

NameError: name 'cloud_model' is not defined

In [32]:
!tree benchmark_src


benchmark_src
├── __pycache__
│   └── main.cpython-38.pyc
└── main.py

1 directory, 2 files


In [ ]:
# Zip submission
!cd unet_src && zip -r ../submission.zip *

In [34]:
!du -h submission.zip


84M	submission.zip
